# Proyecto Henry n° 1
- En esta ocasion se realizara un <b>E.T.L</b>, <b>E.D.A</b> de dos archivos `.csv` a los cuales se le aplicara un modelo de `Machine Learning` para crear un 
<i>Sistema de Recomendacion <i>.

## Importar las librerias a utilizar

In [1]:
import pandas as pd
import numpy as np
import ast
from unidecode import unidecode

### Lectura de los archivos `.csv`.

In [2]:
credits = pd.read_csv('../datasets/credits.csv')
movies = pd.read_csv('../datasets/movies_dataset.csv')

C:\Users\Martin\AppData\Local\Temp\ipykernel_18956\737807593.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('../datasets/movies_dataset.csv')


### Creacion de reporte, presentando una interfaz, acerca de los DataFrames de `movies` y `credits`

## DF movies - ETL
* <p>Se realiza una copia del dataframe (df) como medida de no afectar al original<p>

In [3]:
movies_copia = movies.copy()
movies_copia.isna().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

<p style="color:rgb(190,190,190)">Se puede observar varias columnas con gran cantidad de valores nulos, por ende, seran eliminadas. A su vez, seran eliminadas aquellas columnas que no aporten valor<p>

* Eliminacion de columnas

In [4]:
delete_columns = ["video","imdb_id","adult","original_title","poster_path","homepage","belongs_to_collection"]

movies_copia.drop(columns=delete_columns,inplace=True)

# Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

* Eliminamos posibles duplicados

In [5]:
movies_copia.drop_duplicates(inplace=True)

* Rellenamos valores nulos por 0 en los campos de las columnas <i>revenue</i> y <i>budget</i> 

In [6]:
print(movies_copia[["revenue","budget"]].isna().sum())
movies_copia.fillna(value={"budget":0,"revenue":0},inplace=True)

revenue    6
budget     0
dtype: int64


* Eliminamos valores nulos del campo <i>release</i> con el fin de cambiar de cambiar el tipo de dato a `datetime`

In [7]:
movies_copia.dropna(subset="release_date",inplace=True)
# print(movies_copia["release_date"].info()) #dtype: object
movies_copia["release_date"] = pd.to_datetime(movies_copia["release_date"],format="%Y-%m-%d",errors="coerce").dropna()
# print(movies_copia["release_date"].info()) #dtype: datetime

print(movies_copia["release_date"].isna().sum())

3


* Elimino valores nulos del campo genres

In [8]:
# movies_copia.dropna(subset="genres",inplace=True)

* Creo una columna especifica para el año de lanzamiento

In [9]:
movies_copia["release_year"]=movies_copia["release_date"].dropna().dt.year.astype(int)

* Cambio a dato `int` los valores de la columna <i>id</i>
<p style="color:gray">Primero relazio cambios en registros extraños<p>

In [10]:
movies_copia["id"]=movies_copia["id"].str.replace("-","")
movies_copia["id"] = movies_copia["id"].astype(int)

## Creamos la funcion para desanidar  los campos con diccionarios 

1) DataFrame-movies

In [11]:
# DF-MOVIES
def divide_columns(valores,indice, key=False,value=False):
    
    # cleaned_string = valores.replace(" ", "")
    list_of_dicts = ast.literal_eval(valores) #Toma un string como input y lo evalua como una estructura de python. string ---> lista de diccionaries.
    new_values = []
    if(len(list_of_dicts)==0): return np.nan
    for i in range(len(list_of_dicts)):
        if(value and list_of_dicts[i][key]==value):
            new_values.append(list_of_dicts[i][indice])
        if key==False:
            new_values.append(list_of_dicts[i][indice])
        # name_genre.append(list_of_dicts[i]["name"])
    return list(set(new_values))

id_genre = movies["genres"].apply(lambda x: divide_columns(x,"id"))
name_genre = movies["genres"].apply(lambda x: divide_columns(x,"name"))

**Realizamos el renombre de las columnas modificadas**

In [12]:
id_genre= pd.DataFrame(id_genre).rename(columns={"genres":"id_genre"})
name_genre= pd.DataFrame(name_genre).rename(columns={"genres":"name_genre"})

* Concatenamos las nuevas columnas creadas

In [13]:
movies_copia = pd.concat([movies_copia,id_genre,name_genre],axis=1)
movies_copia.head(1)

,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,id_genre,name_genre
0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862.0,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,1995.0,"[16, 35, 10751]","[Family, Comedy, Animation]"


* Elimino los valores nulos de la columna titulo ya que no serviria de nada recomendar una pelicula sin titulo

In [14]:
print(movies_copia.title.isna().sum())
movies_copia.dropna(subset="title",inplace=True)

103


In [15]:
movies_copia.shape

(45363, 20)

* Obtengo los titulos segun el genero

In [16]:
pd.set_option('future.no_silent_downcasting', True)
r=[]
def gotgenre(valor):
    if(type(valor)==float or valor is None):
        return np.nan
    if "Animation" in valor:
        return True
r = movies_copia["name_genre"].apply(gotgenre)
r = r.replace({None:False})
r.fillna(value=False,inplace=True)

list(movies_copia["title"][r])


['Toy Story',
 'Balto',
 'Pocahontas',
 'A Goofy Movie',
 'Gumby: The Movie',
 'The Swan Princess',
 'The Lion King',
 'The Secret Adventures of Tom Thumb',
 'The Nightmare Before Christmas',
 'The Pagemaster',
 'Aladdin',
 'Snow White and the Seven Dwarfs',
 'Beauty and the Beast',
 'Pinocchio',
 'Heavy Metal',
 'The Aristocats',
 'All Dogs Go to Heaven 2',
 'James and the Giant Peach',
 'Space Jam',
 'Oliver & Company',
 'Institute Benjamenta, or This Dream People Call Human Life',
 'Ghost in the Shell',
 'A Close Shave',
 'The Hunchback of Notre Dame',
 'The Adventures of Pinocchio',
 'The Land Before Time III: The Time of the Great Giving',
 'Cinderella',
 'Winnie the Pooh and the Blustery Day',
 'The Three Caballeros',
 'The Sword in the Stone',
 'So Dear to My Heart',
 'Dumbo',
 "Pete's Dragon",
 'Bedknobs and Broomsticks',
 'Alice in Wonderland',
 'The Fox and the Hound',
 'Aladdin and the King of Thieves',
 'The Wrong Trousers',
 'Faust',
 'A Grand Day Out',
 'Akira',
 'Fantasi

* Se crea la columna Retorno de Inversion (R.O.I.)

In [17]:
# Cheaquear el tipo de datos de ambas columnas
movies_copia[["revenue","budget"]].info() #revenue --> float64 #budget --> budget object
# Cambio el tipo de dato de budget
movies_copia["budget"]=movies_copia["budget"].astype("Float64")

returns = np.where(movies_copia["budget"]==0,np.nan,movies_copia["revenue"]/movies_copia["budget"])
movies_copia["return"] = returns
movies_copia.fillna({"return":0},inplace=True)

<class 'pandas.core.frame.DataFrame'>
Index: 45363 entries, 0 to 45465
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   revenue  45363 non-null  float64
 1   budget   45363 non-null  object 
dtypes: float64(1), object(1)
memory usage: 1.0+ MB


* Se cambio los valores del titulo a minuscula para efectos practicos

In [18]:
movies_copia["title"] = movies_copia["title"].str.lower()

In [19]:
movies_copia[movies_copia["release_date"].dt.month==12]["title"].count()

3784

In [20]:
movies_copia["release_date"].dt.day_name()

0          Monday
1          Friday
2          Friday
3          Friday
4          Friday
           ...   
45460      Monday
45462    Thursday
45463      Friday
45464      Sunday
45465      Friday
Name: release_date, Length: 45363, dtype: object

## DF credits - ETL
* <p>Se realiza una copia del dataframe (df) como medida de no afectar al original<p>

In [21]:
credits_copy = credits.copy()
credits_copy.shape

(45476, 3)

In [22]:
credits_copy["actors_names"] = credits_copy["cast"].apply(lambda x: divide_columns(x,"name"))

In [23]:
credits_copy["actors_names"]

0        [Jim Varney, Tim Allen, Don Rickles, John Ratz...
1        [David Alan Grier, Kirsten Dunst, Lloyd Berry,...
2        [Jack Lemmon, Ann-Margret, Sophia Loren, Daryl...
3        [Wesley Snipes, Whitney Houston, Dennis Haysbe...
4        [George Newbern, Peter Michael Goetz, Kate McG...
                               ...                        
45471          [Leila Hatami, Elham Korda, Kourosh Tahami]
45472    [Roeder, Betty Uy-Regala, Modesta, Angeli Baya...
45473    [Tom Wright, Erika Eleniak, Jeremy Lelliott, J...
45474    [Aleksandr Chabrov, Pavel Pavlov, Nathalie Lis...
45475                                                  NaN
Name: actors_names, Length: 45476, dtype: object

In [24]:
credits_copy["director_names"]=credits_copy["crew"].apply(lambda x: divide_columns(x,"name","job","Director"))

In [25]:
credits_copy["id"].info(), movies_copia["id"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 45476 entries, 0 to 45475
Series name: id
Non-Null Count  Dtype
--------------  -----
45476 non-null  int64
dtypes: int64(1)
memory usage: 355.4 KB
<class 'pandas.core.series.Series'>
Index: 45363 entries, 0 to 45465
Series name: id
Non-Null Count  Dtype  
--------------  -----  
45363 non-null  float64
dtypes: float64(1)
memory usage: 708.8 KB


(None, None)

### Realizo un merge

In [26]:
cast_and_crew = credits_copy[["id","actors_names","director_names"]]

movies_merged = movies_copia.merge(cast_and_crew,on="id",how="inner")

* Normalizacion de titulo a minuscula para facilitar la busqueda

In [27]:
movies_merged_copy = movies_merged.copy()
movies_merged_copy["title"]=movies_merged_copy["title"].str.lower()

* Exporto el csv para posterior evaluacion de la API

In [28]:
movies_merged_copy["popularity"]=movies_merged_copy["popularity"].astype("Float64")

## Guardado
<p>Previo a guardar se eliminan las ultimas columnas a no utilizar</p>

In [29]:
movies_merged_copy.columns

Index(['budget', 'genres', 'id', 'original_language', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'release_year', 'id_genre', 'name_genre',
       'return', 'actors_names', 'director_names'],
      dtype='object')

In [30]:
rec_system = movies_merged_copy[["title","overview","name_genre","actors_names","director_names","tagline","vote_average"]]

In [31]:
movies_merged_copy[movies_merged_copy["vote_average"]==movies_merged_copy["vote_average"].max()]["title"]

186                                      reckless
394                          girl in the cadillac
707      the haunted world of edward d. wood, jr.
737        carmen miranda: bananas is my business
1634                     other voices other rooms
                           ...                   
45009                             the human surge
45158                            first round down
45210                               der herr karl
45321           the many faces of christopher lee
45330       lego dc super hero girls: brain drain
Name: title, Length: 185, dtype: object

In [32]:
rec_system

,title,overview,name_genre,actors_names,director_names,tagline,vote_average
0,toy story,"Led by Woody, Andy's toys live happily in his ...","[Family, Comedy, Animation]","[Jim Varney, Tim Allen, Don Rickles, John Ratz...",[John Lasseter],NaN,7.7
1,jumanji,When siblings Judy and Peter discover an encha...,"[Fantasy, Family, Adventure]","[David Alan Grier, Kirsten Dunst, Lloyd Berry,...",[Joe Johnston],Roll the dice and unleash the excitement!,6.9
2,grumpier old men,A family wedding reignites the ancient feud be...,"[Comedy, Romance]","[Jack Lemmon, Ann-Margret, Sophia Loren, Daryl...",[Howard Deutch],Still Yelling. Still Fighting. Still Ready for...,6.5
3,waiting to exhale,"Cheated on, mistreated and stepped on, the wom...","[Drama, Comedy, Romance]","[Wesley Snipes, Whitney Houston, Dennis Haysbe...",[Forest Whitaker],Friends are the people who let you be yourself...,6.1
4,father of the bride part ii,Just when George Banks has recovered from his ...,[Comedy],"[George Newbern, Peter Michael Goetz, Kate McG...",[Charles Shyer],Just When His World Is Back To Normal... He's ...,5.7
...,...,...,...,...,...,...,...
45418,robin hood,"Yet another version of the classic epic, with ...","[Action, Drama, Romance]","[Uma Thurman, Patrick Bergin, Jeroen Krabbé, D...",[John Irvin],NaN,5.7
45419,century of birthing,An artist struggles to finish his work while a...,[Drama],"[Roeder, Betty Uy-Regala, Modesta, Angeli Baya...",[Lav Diaz],NaN,9.0
45420,betrayal,"When one of her hits goes wrong, a professiona...","[Drama, Action, Thriller]","[Tom Wright, Erika Eleniak, Jeremy Lelliott, J...",[Mark L. Lester],A deadly game of wits.,3.8
45421,satan triumphant,"In a small town live two brothers, one a minis...",NaN,"[Aleksandr Chabrov, Pavel Pavlov, Nathalie Lis...",[Yakov Protazanov],NaN,0.0


In [33]:
rec_system.to_parquet("../datasets/rec_system.parquet")

In [34]:
# movies_merged_copy.to_csv(path_or_buf="../datasets/movies_merged.csv")
movies_merged_copy.to_parquet("movies_merged.parquet")

In [35]:
movies_merged_copy["tagline"]

0                                                      NaN
1                Roll the dice and unleash the excitement!
2        Still Yelling. Still Fighting. Still Ready for...
3        Friends are the people who let you be yourself...
4        Just When His World Is Back To Normal... He's ...
                               ...                        
45418                                                  NaN
45419                                                  NaN
45420                               A deadly game of wits.
45421                                                  NaN
45422                                                  NaN
Name: tagline, Length: 45423, dtype: object

# FUNCIONES 

## Titulo de pelicula

In [36]:
def cantidad_filmaciones_mes(mes):
    try:
        a = movies_merged_copy[movies_merged_copy["release_date"].dt.month_name(locale='es_ES.utf8')== mes.capitalize()]
        return a["title"].count()
    except:
        print("Check your input")
    
cantidad = cantidad_filmaciones_mes("febrero")
cantidad

3036

In [37]:
dias_norm = {
    "Miã©rcoles":"Miercoles",
    "Sã¡bado":"Sabado"
}
dias = movies_merged_copy["release_date"].dt.day_name(locale='es_Es.utf8')
dias= dias.replace(dias_norm)
# normalized_days = dias.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('ascii')
dias.unique()

array(['Lunes', 'Viernes', 'Jueves', 'Miercoles', 'Sabado', 'Martes',
       'Domingo'], dtype=object)

In [38]:

def cantidad_filmaciones_dia(dia):
    try:
        a = movies_merged_copy[dias== unidecode(dia).capitalize()]
        return a["title"].count()
    except:
        print("Check your input")
    
cantidad = cantidad_filmaciones_dia("sabado")
cantidad

5156

In [39]:
def popularidad_titulo (titulo):
    titulo=titulo.lower()
    df_title = movies_merged_copy[movies_merged_copy["title"]==titulo]
    popularity = list(df_title["popularity"])[0]
    return print(f"""La película {titulo} fue estrenada en el año {int(list(df_title["release_year"])[0])}
                  cuenta con una popularidad de {popularity}""")
    
popularidad_titulo("tOy story")

La película toy story fue estrenada en el año 1995
                  cuenta con una popularidad de 21.946943


In [40]:
def votos_titulo (titulo):
    titulo=titulo.lower()
    df_title = movies_merged_copy[movies_merged_copy["title"]==titulo]
    cant_votos = list(df_title["vote_count"])[0]
    if(cant_votos<2000): 
        return print(f"Lo siento, {titulo} cuenta solo con {cant_votos}, debes elegir una pelicula con almenos 2000")
    return print(f"""La película {titulo} fue estrenada en el año {int(list(df_title["release_year"])[0])}
                  La misma cuenta con un total de {cant_votos} valoraciones, con un promedio de {list(df_title["vote_average"])[0]}""")
    
votos_titulo("tOy story")

La película toy story fue estrenada en el año 1995
                  La misma cuenta con un total de 5415.0 valoraciones, con un promedio de 7.7


In [ ]:
# for i in movies_merged_copy["actors_names"]:

for i 

In [41]:
def get_actor(nombre):
    cuenta_movies = 0
    cuenta_dinero = 0
    for nombres,i in zip(movies_merged_copy["actors_names"],range(movies_merged_copy.shape[0])):
        # if(type(nombres) in [float,int] ): continue
        if(isinstance(nombres,list) and nombre in nombres):
            cuenta_movies +=1
            cuenta_dinero += movies_merged_copy["return"].iloc[i]
    promedio = cuenta_dinero/cuenta_movies
    return print(f"El actor {nombre} participo en {cuenta_movies} consiguiendo un total de {round(cuenta_dinero,2)} mil dolares, con un promedio de {round(promedio,2)} mil dolares por pelicula")
    
get_actor("Tom Hanks")

El actor Tom Hanks participo en 71 consiguiendo un total de 178.85 mil dolares, con un promedio de 2.52 mil dolares por pelicula


In [42]:
movies_merged_copy["director_names"].fillna("[]",inplace=True)

C:\Users\Martin\AppData\Local\Temp\ipykernel_18956\3256321663.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  movies_merged_copy["director_names"].fillna("[]",inplace=True)


In [43]:
def get_director(nombre):

    nombre = nombre.title()
    peliculas_return = {}
    cuenta_dinero_total = 0

    for nombres,i in zip(movies_merged_copy["director_names"],range(movies_merged_copy.shape[0])):

        if(isinstance(nombres,list) and nombre in nombres):

            movie_title= movies_merged_copy["title"].str.capitalize().iloc[i]
            individual_return = movies_merged_copy["return"].iloc[i]
            release_year = movies_merged_copy["release_year"].iloc[i]
            budget = movies_merged_copy["budget"].iloc[i]
            revenue = movies_merged_copy["revenue"].iloc[i]

            peliculas_return[movie_title] = {"retorno": round(individual_return,2),
                                            "año_lanzamiento": int(release_year),
                                            "costo":budget,
                                            "revenue":revenue}
            cuenta_dinero_total += movies_merged_copy["return"].iloc[i]

    print(peliculas_return)
    return print(f"El director {nombre} consiguio un total de {round(cuenta_dinero_total,2)} mil dolares")
    
get_director("John Lasseter")


{'Toy story': {'retorno': 12.45, 'año_lanzamiento': 1995, 'costo': 30000000.0, 'revenue': 373554033.0}, "A bug's life": {'retorno': 3.03, 'año_lanzamiento': 1998, 'costo': 120000000.0, 'revenue': 363258859.0}, 'Toy story 2': {'retorno': 5.53, 'año_lanzamiento': 1999, 'costo': 90000000.0, 'revenue': 497366869.0}, 'Luxo jr.': {'retorno': 0.0, 'año_lanzamiento': 1986, 'costo': 0.0, 'revenue': 0.0}, 'Cars': {'retorno': 3.85, 'año_lanzamiento': 2006, 'costo': 120000000.0, 'revenue': 461983149.0}, 'Cars 2': {'retorno': 2.8, 'año_lanzamiento': 2011, 'costo': 200000000.0, 'revenue': 559852396.0}, 'Tin toy': {'retorno': 0.0, 'año_lanzamiento': 1988, 'costo': 0.0, 'revenue': 0.0}, "Red's dream": {'retorno': 0.0, 'año_lanzamiento': 1987, 'costo': 0.0, 'revenue': 0.0}, 'Knick knack': {'retorno': 0.0, 'año_lanzamiento': 1989, 'costo': 0.0, 'revenue': 0.0}, 'Mater and the ghostlight': {'retorno': 0.0, 'año_lanzamiento': 2006, 'costo': 0.0, 'revenue': 0.0}}
El director John Lasseter consiguio un tota

In [44]:
movies_merged_copy.shape

(45423, 23)